In [1]:
import os
import pandas as pd
import numpy as np
from pathlib import Path

if not "root" in locals():
    current_path = Path(os.getcwd())
    root = current_path.parent.absolute()

In [2]:
os.chdir(root)

from tools.add_AR_cols import add_AR_cols
from data_treatment.box_creator import create_binary_box
PATH = "../"

df = pd.read_csv('database.csv')

df = create_binary_box(df, relative_threshold = 0.05, box_length=5).dropna(axis = 0)
df = add_AR_cols(df,7)



In [3]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_selection import SelectFromModel

# prescale_X = df.drop(columns="Box")
# y = df["Box"]
# X = sm.add_constant((prescale_X - prescale_X.mean())/prescale_X.std())


# lsvc = LogisticRegression(penalty="l1", solver="liblinear", C = 0.01).fit(X, y)
# model = SelectFromModel(lsvc, prefit=True)
# X_new = model.transform(X)
# X_new.shape

In [4]:
# X.columns[model.get_support()]

Analyse des features pertinentes en fonction du temps

In [57]:
window = 1000  #Selection des meilleures colonnes par fenêtres de 1 an glissantes
step = 5
C = 0.05

In [58]:
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

tab_coeff = []

y = df['Box']
X = df.drop(['Box', 'Neutral_SENTIMENT', 'Bull-Bear Spread_SENTIMENT', "Date"], axis=1)
X = (X - X.mean())/X.std()


for i in tqdm(range(0, df.shape[0] - window, step)):
    
    
    sub_X = X.iloc[i : i + window]
    sub_y = y.iloc[i : i + window]
    
    
    lasso = LogisticRegression(C=C, fit_intercept=True, penalty='l1', solver='saga', max_iter = 4000)
    lasso.fit(sub_X,sub_y)
    tab_coeff.append([df["Date"].iloc[i]] + list(lasso.coef_[0]) )

100%|██████████| 539/539 [01:15<00:00,  7.12it/s]


In [59]:
tab_coeff = np.array([tab_coeff])[0]
tab_coeff.shape

(539, 164)

In [60]:
to_graph = pd.DataFrame(tab_coeff, columns = ["Date"] + list(X.columns))

nb_significant = 0
for col in tqdm(to_graph.columns):
    if col != "Date":
        
        
        to_graph[col] = to_graph[col].astype(float)
        
        
        if ((to_graph[col] != 0).sum() == 0):
            to_graph = to_graph.drop(columns = [col])
        else:
            nb_significant += 1
            
nb_significant

100%|██████████| 164/164 [00:00<00:00, 1818.78it/s]


87

In [61]:
if True:
    scores = []
    K = 10

    for col in to_graph.columns[1:]:
        scores.append(to_graph[col].abs().sum())
        
    scores = np.array(scores)    
    ind = np.argpartition(scores, -K)[-K:]
        
    cols = to_graph.columns[1:][ind]
    
else:
    cols = to_graph.columns

In [62]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"


fig = px.line(to_graph, x = "Date" , y = cols)
fig.show()